# 교차 검증 Cross Validation
모델의 성능을 신뢰할 수 있게 평가하는 방법 중 하나로, 훈련셋을 여러개로 나누어 일부를 검증셋으로 사용

- 모델의 과적합을 탐지/방지하는데 효과가 있음

## K-Fold


주어진 데이터를 단순히 균등하게 분할하는 방법이다. K는 분할할 fold(세트)의 수를 나타낸다.


데이터는 무작위로 섞인 후, 각 fold로 동일하게 나눠지며, 각 fold가 한 번씩 검증 세트가 되고 나머지 fold는 훈련 세트가 된다.


중요한 점은 이 방법에서는 클래스 레이블의 분포를 고려하지 않는다.


따라서 데이터가 불균형할 경우, 검증 세트나 훈련 세트의 클래스 분포가 편향될 수 있다.


![](https://d.pr/i/0pWjyI+)

In [1]:
from statistics import LinearRegression

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# iris 데이터셋
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_validate

iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42, stratify=iris.target)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(120, 4) (120,)
(30, 4) (30,)


In [3]:
from sklearn.linear_model import LogisticRegression # 분류
from sklearn.model_selection import KFold
# KFold : 데이터를 여러 조각으로 나누어 교차검증할 때 쓰는 도구
from sklearn.metrics import accuracy_score, f1_score

lr_clf = LogisticRegression(max_iter=1000)
kfold = KFold(n_splits=5)
cv_accuracy = [] # 정확도 관리

for train_index, val_index in kfold.split(X_train):
    # 학습/검증 분리된 인덱스
    # print(train_index)
    # print(val_index)

    # 실제 학습/ 검증 데이터
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
    print(pd.Series(y_train_fold).value_counts())
    print(pd.Series(y_val_fold).value_counts())

    # 모델 학습
    lr_clf.fit(X_train_fold, y_train_fold)

    # 모델 예측
    y_pred = lr_clf.predict(X_val_fold)

    # 평가
    # accuracy_score(정답, 예측값)
    acc_val = accuracy_score(y_val_fold, y_pred)
    cv_accuracy.append(acc_val)

print('검증점수 : ',  np.mean(cv_accuracy))


# 학습 후 평가
acc_score = lr_clf.score(X_test, y_test)
print("평가 점수 : ", acc_score)


0    33
1    32
2    31
Name: count, dtype: int64
2    9
1    8
0    7
Name: count, dtype: int64
1    33
2    32
0    31
Name: count, dtype: int64
0    9
2    8
1    7
Name: count, dtype: int64
2    33
1    32
0    31
Name: count, dtype: int64
0    9
1    8
2    7
Name: count, dtype: int64
0    33
1    33
2    30
Name: count, dtype: int64
2    10
0     7
1     7
Name: count, dtype: int64
2    34
0    32
1    30
Name: count, dtype: int64
1    10
0     8
2     6
Name: count, dtype: int64
검증점수 :  0.9666666666666668
평가 점수 :  0.9666666666666667


In [4]:
# StratifiedKFold
# - 학습/ 검증 셋을 나눌 때, 타겟의 비율을 유지한 채 분할
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier


dt_clf = DecisionTreeClassifier(random_state=42) # 트리 분할을 통해 실제 분류 작업
skfold = StratifiedKFold(n_splits=5)
cv_accuracy = []

# 학습에 쓸 행 번호 # 검증에 쓸 행 번호
for train_index, val_index in skfold.split(X_train, y_train) :


     # 실제 학습/ 검증 데이터
     # 4조각          1조각
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
    print(pd.Series(y_train_fold).value_counts())
    print(pd.Series(y_val_fold).value_counts())
    print()

    # 모델 학습
    dt_clf.fit(X_train_fold, y_train_fold)

    # 평가
    # accuracy_score(정답, 예측값)
    acc_score = dt_clf.score(X_val_fold, y_val_fold)
    cv_accuracy.append(acc_score)


print(cv_accuracy)
print('검증점수 : ',  np.round(np.mean(cv_accuracy), 4))
print("평가 점수 : ", np.round(dt_clf.score(X_test, y_test), 4))

2    32
1    32
0    32
Name: count, dtype: int64
0    8
2    8
1    8
Name: count, dtype: int64

0    32
2    32
1    32
Name: count, dtype: int64
2    8
1    8
0    8
Name: count, dtype: int64

0    32
2    32
1    32
Name: count, dtype: int64
2    8
0    8
1    8
Name: count, dtype: int64

0    32
2    32
1    32
Name: count, dtype: int64
0    8
2    8
1    8
Name: count, dtype: int64

0    32
2    32
1    32
Name: count, dtype: int64
2    8
0    8
1    8
Name: count, dtype: int64

[0.9166666666666666, 0.9583333333333334, 0.9583333333333334, 0.9583333333333334, 0.9166666666666666]
검증점수 :  0.9417
평가 점수 :  0.8667


# cross_val_score()

In [5]:
from sklearn.model_selection import cross_val_score
# - estimator : 모델 객체
# - X : 학습 데이터
# - y : 학습 데이터 입력
# - cv : 반복횟수
# - scoring : 평가지표
classifier = DecisionTreeClassifier(random_state=42) # 이 모델은 아직 학습이 안 된 상태여서 스코어를 하면 안된다. -> cross_val_score를 해주고 다시 fit 해주는 과정 필요

# cv = 5 : KFold(n_splits=5)
skfold = StratifiedKFold(n_splits=5)

# cross_val_score()에서는 classifier의 복사본 학습
acc_scores = cross_val_score(classifier, X_train, y_train, cv=skfold,
                             scoring='accuracy')
print('accuray 검증점수 : ', np.round(np.mean(acc_scores), 4))


acc_scores = cross_val_score(classifier, X_train, y_train, cv=skfold,
                             scoring='f1_macro') # 클래스별 F1의 단순 평균
print('f1_macro 검증점수 : ', np.round(np.mean(acc_scores), 4))

acc_scores = cross_val_score(classifier, X_train, y_train, cv=skfold,
                             scoring='f1_micro') # 전체 TP/FP/FN으로 계산
print('f1_micro 검증점수 : ', np.round(np.mean(acc_scores), 4))

acc_scores = cross_val_score(classifier, X_train, y_train, cv=skfold,
                             scoring='f1_weighted') # 클래스 샘플 수로 가중 평균
print('f1_weighted 검증점수 : ', np.round(np.mean(acc_scores), 4))



# NotFittedError : 원본 모델 학습은 별도로 진행해야 한다.
classifier.fit(X_train, y_train)     # 내부적으로 score 하려면 학습된 모델이 필요해서
print('평가점수 : ', np.round(classifier.score(X_test, y_test), 4)) # not fitted error



accuray 검증점수 :  0.9417
f1_macro 검증점수 :  0.9413
f1_micro 검증점수 :  0.9417
f1_weighted 검증점수 :  0.9413
평가점수 :  0.9333


## cross_validate
교차검증 수행하면서, 여러 성능지표 계산 및 훈련/예측 시간까지 측정

In [6]:
# 붓꽃 데이터셋
from sklearn.datasets import load_iris
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import cross_validate, StratifiedKFold, train_test_split
from sklearn.multiclass import OneVsRestClassifier

iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.2, random_state=42, stratify=iris.target)


# solver = 'liblinear' : 소규모 데이터셋에 적합한 작동 방식
# OneVsRestClassifier : 여러개의 이진분류 모델을 만들어 다중 분류가 가능하게 함
classifier = OneVsRestClassifier(
    LogisticRegression(max_iter=1000, solver='liblinear'))


skfold = StratifiedKFold(n_splits=5)

# scoring = 'f1_score' 사용하려면, 평균 구하는 방식을 같이 지정해야한다.
# make_scorer : 스코어러 객체로 바꿔주는 함수
f1_macro = make_scorer(f1_score, average='macro')

# cross_validate : cross_val_score 보다 업그레이드된 버전으로
                    # 교차검증을 실행 후 점수, 시간, 모델 반환
result = cross_validate(classifier, X_train, y_train, cv=skfold, scoring=f1_macro,
                       return_estimator=True) # 학습된 모델객체 5개가 들어감
print(result.keys())


scores = result['test_score']
print(scores)
print('검증점수(f1) : ', np.mean(scores))

# 교차검증에서 제일 성능이 좋았던 폴드의 모델 뽑기
estimators = result['estimator'] # 학습된 모델 객체 5개
best_score_index = scores.argmax() # 최대값의 인덱스
# print(best_score_index)

best_estimator = estimators[best_score_index]

# 예측
print('평가점수(f1) : ', f1_score(y_test, best_estimator.predict(X_test),
      average='macro'))

dict_keys(['fit_time', 'score_time', 'estimator', 'test_score'])
[1.         0.91534392 0.91534392 0.91666667 1.        ]
검증점수(f1) :  0.9494708994708994
평가점수(f1) :  0.9665831244778612


In [7]:
# 생선 다중분류
fish_df = pd.read_csv('https://bit.ly/fish_csv_data')
fish_df

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340
...,...,...,...,...,...,...
154,Smelt,12.2,12.2,13.4,2.0904,1.3936
155,Smelt,13.4,12.4,13.5,2.4300,1.2690
156,Smelt,12.2,13.0,13.8,2.2770,1.2558
157,Smelt,19.7,14.3,15.2,2.8728,2.0672


In [8]:
# 7종류 생선 다중분류
fish_df['Species'].value_counts()

Species
Perch        56
Bream        35
Roach        20
Pike         17
Smelt        14
Parkki       11
Whitefish     6
Name: count, dtype: int64

In [9]:
# 데이터 분할
X = fish_df.drop("Species", axis=1)
y = fish_df["Species"]
X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.2, random_state=42, stratify=y)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(127, 5) (127,)
(32, 5) (32,)


In [10]:
from sklearn.model_selection import cross_val_score
# cross_val_score() 이용
classifier = DecisionTreeClassifier(random_state=42)
skfold = StratifiedKFold(n_splits=5)

# 검증점수
acc_scores = cross_val_score(classifier, X_train, y_train, cv=skfold, scoring='accuracy')
print(acc_scores)
print('검증점수 : ', np.round(np.mean(acc_scores), 4))


# 평가점수
classifier.fit(X_train, y_train)
print('평가점수 : ', np.round(classifier.score(X_test, y_test), 4))

[0.57692308 0.69230769 0.72       0.72       0.8       ]
검증점수 :  0.7018
평가점수 :  0.7812


In [11]:
# cross_validate() 이용
# 데이터 분할
X = fish_df.drop("Species", axis=1)
y = fish_df["Species"]
X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


classifier = OneVsRestClassifier( # 이진분류만 가능한데 다중뷴류도 가능하게끔 해줌
    LogisticRegression(max_iter=1000, solver='liblinear'))


skfold = StratifiedKFold(n_splits=5)
# f1_macro = make_scorer(f1_score, average='macro')

result = cross_validate(classifier, X_train, y_train, cv=skfold, scoring='accuracy',
                       return_estimator=True)
print(result.keys())

# 검증 점수
scores = result['test_score']
print(scores)
print('검증점수(f1) : ', np.mean(scores))

# 교차검증에서 제일 성능이 좋았던 폴드의 모델 뽑기
estimators = result['estimator'] # 내부적으로 이미 학습된 친구
best_score_index = scores.argmax() # 최대값의 인덱스
best_estimator = estimators[best_score_index]


# 평가 점수
print('평가점수(f1) : ', best_estimator.score(X_test, y_test))

(127, 5) (127,)
(32, 5) (32,)


C:\Users\TJ\anaconda3\envs\ai_env2\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


dict_keys(['fit_time', 'score_time', 'estimator', 'test_score'])
[0.96153846 0.92307692 1.         0.96       0.88      ]
검증점수(f1) :  0.944923076923077
평가점수(f1) :  0.90625


### 회귀모델 교차 검증


**scoring 매개변수**
1. **R^2 (결정계수)**: `scoring='r2'`로 설정. 모델의 설명력을 나타내며, 1에 가까울수록 좋음.
2. **MAE (평균 절대 오차)**: `scoring='neg_mean_absolute_error'`로 설정. 절대 오차의 평균을 나타내며, 값이 작을수록 좋음.
3. **MSE (평균 제곱 오차)**: `scoring='neg_mean_squared_error'`로 설정. 제곱 오차의 평균을 나타내며, 값이 작을수록 좋음.
4. **RMSE (평균 제곱근 오차)**: `scoring='neg_root_mean_squared_error'`로 설정. 제곱근 오차의 평균을 나타내며, 값이 작을수록 좋음.
5. **Explained Variance Score**: `scoring='explained_variance'`로 설정. 설명된 분산 비율로, 1에 가까울수록 좋음.

In [12]:
from sklearn.datasets import fetch_california_housing
cal_housing = fetch_california_housing()

cal_housing_df = pd.DataFrame(data=cal_housing.data, columns=cal_housing.feature_names)
cal_housing_df['MedHouseVal'] = cal_housing.target # 10만 달러 단위 4.526 * 10만 = 45만 2600달러

cal_housing_df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


#### 캘리포니아 주택가격 예측


| 특성 이름        | 설명                          |                                                                               |
| ------------ | --------------------------- | ----------------------------------------------------------------------------- |
| `MedInc`     | 블록 그룹의 중간 소득 (단위: \$10,000) |                                                                               |
| `HouseAge`   | 주택의 중간 연령 (년)               |                                                                               |
| `AveRooms`   | 가구당 평균 방 수                  |                                                                               |
| `AveBedrms`  | 가구당 평균 침실 수                 |                                                                               |
| `Population` | 블록 그룹의 인구 수                 |                                                                               |
| `AveOccup`   | 가구당 평균 거주자 수                |                                                                               |
| `Latitude`   | 블록 그룹의 위도                   |                                                                               |
| `Longitude`  | 블록 그룹의 경도                   |           |


In [21]:
from sklearn.linear_model import LinearRegression
# 주택 중위가격 회귀 예측에 대한 교차검증

X, y = fetch_california_housing(return_X_y=True) # data, target 속성만 반환
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 회귀모델
# import 시alt + enter -> 현재 셀에 가져오기
regressor = LinearRegression()

# 평가지표별 교차검증
r2_scores = cross_val_score(regressor, X_train, y_train, cv=5, scoring='r2')
print(r2_scores)

neg_mse_scores = cross_val_score(regressor, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(neg_mse_scores) # -0.50229277 얘가 젤 좋은 거

neg_mae_scores = cross_val_score(regressor, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
print(neg_mae_scores) # mse나 mae나 비슷하다. 예측이 거의 잘 됐다.

neg_rmse_scores = cross_val_score(regressor, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
print(neg_mse_scores)

ex_var_scores = cross_val_score(regressor, X_train, y_train, cv=5,
 scoring='explained_variance')
print(ex_var_scores)


[0.62011512 0.61298876 0.6134416  0.61069973 0.60017477]
[-0.52006533 -0.50229277 -0.52054451 -0.50789718 -0.54552622]
[-0.53113061 -0.52675118 -0.52419835 -0.5247937  -0.5384344 ]
[-0.52006533 -0.50229277 -0.52054451 -0.50789718 -0.54552622]
[0.62017763 0.61302322 0.61344193 0.61071689 0.60018436]
